# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


In [31]:
%pip install xlwings

Note: you may need to restart the kernel to use updated packages.


In [32]:
import csv
import pandas as pd
import xlwings as xw
import numpy as np

## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [33]:
wb = xw.Book('себестоимостьА_в1.xlsx')


In [34]:
sh = wb.sheets['Рецептура']
recipe1 = sh.range('рцп_пшеничный_хлеб').value
sh.range('G5:O10').options(pd.DataFrame).value

,Пшеничная мука,Дрожжи,Сахар,Соль,Куркума молотая,Топленное масло,Кунжутные семена,Масло оливковое
Вода,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.20,0.40,0.020,0.02,0.010,0.005,0.10,NaN,NaN
0.07,0.25,0.010,0.06,0.015,NaN,0.06,0.05,NaN
0.12,0.22,0.005,NaN,0.009,0.005,NaN,0.05,0.03
0.12,0.23,0.010,NaN,0.015,NaN,NaN,NaN,0.05


In [35]:
product = sh.range('G7:O10').options(np.array).value
product

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [36]:
price = sh.range('G14:O14').options(np.array).value
price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [37]:
mult = product * price
mean = [np.nansum(row) for row in mult]
mean

[21.48, 16.525, 17.423000000000002, 18.085]

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [38]:
sh.range('P7:P10').options(transpose=True).value = mean

In [39]:
sh.range('P5').value = "Себестоимость"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [40]:
sh.range('P5').api.WrapText = True
sh.range('P5').api.Font.Bold = True

4. Выполнить 3 с помощью "протягиваемых" формул.

In [11]:
wb.save()
wb.close()

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [41]:
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

recipes = pd.read_csv('recipes_sample.csv')

recipes = recipes[['name', 'id', 'minutes', 'submitted', 'description', 'n_ingredients']]
display(reviews.loc[reviews['recipe_id'] == 44123])

,user_id,recipe_id,date,rating,review
920960,743566,44123,2008-01-28,5,I lived in San Diego for 19 years and would g...
920958,76503,44123,2003-06-03,5,This soup is the Bomb! Don't hesitate to try.....
920957,34206,44123,2003-03-23,5,I just can't say enough about how wonderful th...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [43]:
reviews_df = pd.read_csv('reviews_sample.csv', index_col=0)
recipes_df = pd.read_csv('recipes_sample.csv', index_col=0)


reviews_sample = reviews_df.sample(frac=0.05, random_state=42)
recipes_sample = recipes_df.sample(frac=0.05, random_state=42)


with pd.ExcelWriter('recipes.xlsx') as writer:
    reviews_sample.to_excel(writer, sheet_name='Отзывы')
    recipes_sample.to_excel(writer, sheet_name='Рецепты')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [44]:

wb = xw.Book('recipes.xlsx')


sheet = wb.sheets['Рецепты']


sheet.range('I1').value = 'seconds_assign'

time_minutes = sheet.range('C2').expand('down').value

time_seconds = [time * 60 for time in time_minutes]

sheet.range('I2').options(transpose=True).value = time_seconds

wb.save()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [45]:

sheet.range('J1').value = 'seconds_formula'


row_count = sheet.range('C2').expand('down').count
for i in range(2, row_count+2):
    sheet.range(f'J{i}').formula = f'=C{i}*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [46]:
sheet.range('I1').api.Font.Bold = True
sheet.range('J1').api.Font.Bold = True
sheet.range('I1').api.HorizontalAlignment = -4108  # -4108 соответствует центрированию в Excel
sheet.range('J1').api.HorizontalAlignment = -4108 
wb.save()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [47]:
for index, row in enumerate(sheet.range(f'C2:C1501').value, 2):
    if row < 5:
        sheet.range(f'C{index}').color = (0, 255, 0)
    elif 5 <= row <= 10:
        sheet.range(f'C{index}').color = (255, 255, 0)
    else:
        sheet.range(f'C{index}').color = (255, 0, 0)


## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot)